In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers


c:\Users\rupde\Downloads\VS\.venv\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [2]:
dataset_path = "."


In [3]:
image_size = (224, 224)
batch_size = 32


In [4]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=image_size,
    batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=image_size,
    batch_size=batch_size
)


Found 43994 files belonging to 40 classes.
Using 35196 files for training.
Found 43994 files belonging to 40 classes.
Using 8798 files for validation.


In [5]:
class_names = train_ds.class_names
num_classes = len(class_names)

print(num_classes)


40


In [6]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.map(lambda x, y: (x / 255.0, y))
val_ds   = val_ds.map(lambda x, y: (x / 255.0, y))

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds   = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


In [7]:
model = tf.keras.Sequential([
    layers.Input(shape=(224, 224, 3)),
    layers.Conv2D(32, 3, activation="relu"),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation="relu"),
    layers.MaxPooling2D(),
    layers.Conv2D(128, 3, activation="relu"),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation="softmax")
])


In [8]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)


In [9]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)


Epoch 1/10
1100/1100 ━━━━━━━━━━━━━━━━━━━━ 316s 262ms/step - accuracy: 0.4906 - loss: 1.7361 - val_accuracy: 0.7170 - val_loss: 0.8963
Epoch 2/10
1100/1100 ━━━━━━━━━━━━━━━━━━━━ 299s 272ms/step - accuracy: 0.6791 - loss: 1.0020 - val_accuracy: 0.7855 - val_loss: 0.6385
Epoch 3/10
1100/1100 ━━━━━━━━━━━━━━━━━━━━ 275s 250ms/step - accuracy: 0.7430 - loss: 0.7783 - val_accuracy: 0.8316 - val_loss: 0.4976
Epoch 4/10
1100/1100 ━━━━━━━━━━━━━━━━━━━━ 275s 250ms/step - accuracy: 0.7838 - loss: 0.6458 - val_accuracy: 0.8483 - val_loss: 0.4328
Epoch 5/10
1100/1100 ━━━━━━━━━━━━━━━━━━━━ 273s 248ms/step - accuracy: 0.8130 - loss: 0.5458 - val_accuracy: 0.8701 - val_loss: 0.3882
Epoch 6/10
1100/1100 ━━━━━━━━━━━━━━━━━━━━ 272s 247ms/step - accuracy: 0.8334 - loss: 0.4886 - val_accuracy: 0.8751 - val_loss: 0.3645
Epoch 7/10
1100/1100 ━━━━━━━━━━━━━━━━━━━━ 276s 251ms/step - accuracy: 0.8536 - loss: 0.4252 - val_accuracy: 0.8757 - val_loss: 0.3749
Epoch 8/10
1100/1100 ━━━━━━━━━━━━━━━━━━━━ 270s 245ms/step - ac

In [10]:
final_train_acc = history.history["accuracy"][-1]
final_val_acc = history.history["val_accuracy"][-1]

print("Final Training Accuracy:", final_train_acc)
print("Final Validation Accuracy:", final_val_acc)


Final Training Accuracy: 0.8929992318153381
Final Validation Accuracy: 0.8854284882545471


In [11]:
model.save("crop_disease_cnn_model.keras")


In [12]:
import tensorflow as tf

model = tf.keras.models.load_model("crop_disease_mobilenet.keras")


ValueError: File not found: filepath=crop_disease_mobilenet.keras. Please ensure the file is an accessible `.keras` zip file.

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

img_path = "test_leaf.jpg"

img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = img_array / 255.0
img_array = np.expand_dims(img_array, axis=0)

predictions = model.predict(img_array)
predicted_class = class_names[np.argmax(predictions)]

print("Predicted Disease:", predicted_class)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 397ms/step
Predicted Disease: Strawberry___Leaf_scorch


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models

base_model = MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights="imagenet"
)

base_model.trainable = False  # freeze pretrained layers

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation="softmax")
])


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [ ]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)


In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
)


Epoch 1/5
925/925 ━━━━━━━━━━━━━━━━━━━━ 266s 285ms/step - accuracy: 0.6388 - loss: 1.1755 - val_accuracy: 0.9088 - val_loss: 0.2977
Epoch 2/5
925/925 ━━━━━━━━━━━━━━━━━━━━ 272s 295ms/step - accuracy: 0.8580 - loss: 0.4302 - val_accuracy: 0.9230 - val_loss: 0.2233
Epoch 3/5
925/925 ━━━━━━━━━━━━━━━━━━━━ 280s 303ms/step - accuracy: 0.8826 - loss: 0.3497 - val_accuracy: 0.9370 - val_loss: 0.1868
Epoch 4/5
925/925 ━━━━━━━━━━━━━━━━━━━━ 271s 293ms/step - accuracy: 0.8908 - loss: 0.3131 - val_accuracy: 0.9410 - val_loss: 0.1773
Epoch 5/5
925/925 ━━━━━━━━━━━━━━━━━━━━ 274s 296ms/step - accuracy: 0.9041 - loss: 0.2834 - val_accuracy: 0.9413 - val_loss: 0.1628


In [ ]:
model.save("crop_disease_mobilenet.keras")
